In [23]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **CREATING A ORDER INSTANCE**

In [29]:
import random
customer_id = []

def RandomIDGenerator(user_id:str):
    ids = str(random.randint(1000000,99999999)) + '-' + user_id
    customer_id.append(ids)
    return ids

In [90]:
from enum import Enum
from time import time
asks = []
bids = []

class Side(Enum):
    BUY = 1
    SELL = 0


class Order(object):
    def __init__(self,order_id:str):
        self.order_id = order_id
        self.time = int(1e6 * time())

    def __getType__(self):
        return self.__class__
    

class CancelOrder(Order):
    def __init__(self,order_id):
        super().__init__(order_id)

    def repr(self):
        return "Cancel Order: {}.".format(self.order_id)
    

class MarketOrder(Order):
    def __init__(self,order_id:str,side:Side,size:int):
        super().__init__(order_id)
        self.side = side
        self.size = self.remainingToFill = size


    def __repr__(self):
        return "Market Order: {1} unit(s).".format(
            "BUY" if self.side == "BUY" else "SELL",
            self.remainingToFill)
        

class LimitOrder(Order):
    def __init__(self,order_id:str,side:Side,size:int,price:int):
        super().__init__(order_id)
        self.size = size
        self.side = side
        self.price = price
        self.size = self.remainingToFill = size

    def __lt__(self, other):
        if self.price != other.price:
            if self.side == Side.BUY:
                return self.price > other.price
            else:
                return self.price < other.price

        elif self.time != other.time:
             return self.time < other.time

        elif self.size != other.size:
            self.size < other.size

    def __repr__(self):
        return 'Limit Order: {1} unit(s) at {2}.'.format(
            "BUY" if self.side == "BUY" else "SELL",
            self.remainingToFill, self.price)

# **CREATING A TRADE INSTANCE**

In [94]:
Trade_book = []
Trade_time_book = []
order_id = []
book_price = []
class Trade(object):

    def __init__(self,incoming_side:Side,price:float,trade_size:int,incoming_order_id:int,book_order_id:str):
        self.timestamp = int(1e6 * time())
        self.side = incoming_side
        self.price = price
        self.size = trade_size
        self.incoming_order_id = incoming_order_id
        self.book_order_id = book_order_id
        Trade_book.append('Executed: {0} {1} units at {2} ///@ id no. {3}'.format(self.side, self.size, self.price,self.book_order_id))
        Trade_time_book.append(time())
        order_id.append(self.book_order_id)

    def __repr__(self):
        book_price.append(self.price)
        return print('Executed: {0} {1} units at {2}'.format(self.side, self.size, self.price))
    

# **ORDER BOOK CODE**

In [78]:
from sortedcontainers import SortedList
from typing import List, Union
from time import time


In [79]:
Price = []
Time = []
Canceled_trade_id = []
class Orderbook(object):
    
    def __init__(self):
        self.bids : SortedList[Order] = SortedList()
        self.asks : SortedList[Order] = SortedList()
        self.trades = []
        
    def processOrder(self,incomingOrder):
        
        if incomingOrder.__class__ == CancelOrder:
            for order in self.bids:
                if incomingOrder.order_id == order.order_id:
                    Canceled_trade_id.append(incomingOrder.order_id)
                    self.bids.discard(order)
                    break
            
            for order in self.asks:
                if incomingOrder.order_id == order.order_id:
                    Canceled_trade_id.append(incomingOrder.order_id)
                    self.asks.discard(order)
                    break
            
            return "Exiting Process Order"
        
        
        def whileClause():
            
            if incomingOrder.side==Side.BUY:
                if incomingOrder.__class__ == LimitOrder:
                    return len(self.asks) > 0 and incomingOrder.price >= self.asks[0].price # Limit order on the BUY side
                elif incomingOrder.__class__ == MarketOrder:
                    return len(self.asks) > 0 # Market order on the BUY side
            else:
                if incomingOrder.__class__ == LimitOrder:
                    return len(self.bids) > 0 and incomingOrder.price <= self.bids[0].price # Limit order on the SELL side
                elif incomingOrder.__class__ == MarketOrder:
                    return len(self.bids) > 0 # Market order 
                
        while whileClause():
            bookOrder = None
            if incomingOrder.side==Side.BUY:
                bookOrder = self.asks.pop(0)
            else:
                bookOrder = self.bids.pop(0)

            if incomingOrder.remainingToFill == bookOrder.remainingToFill:  # if the same volume
                volume = incomingOrder.remainingToFill
                incomingOrder.remainingToFill -= volume
                bookOrder.remainingToFill -= volume
                self.trades.append(Trade(
                    incomingOrder.side, bookOrder.price, volume, incomingOrder.order_id, bookOrder.order_id))
                break

            elif incomingOrder.remainingToFill > bookOrder.remainingToFill:  # incoming has greater volume
                volume = bookOrder.remainingToFill
                incomingOrder.remainingToFill -= volume
                bookOrder.remainingToFill -= volume
                self.trades.append(Trade(
                    incomingOrder.side, bookOrder.price, volume, incomingOrder.order_id, bookOrder.order_id))
                print(f"PRICE IS {bookOrder.price} AT TIME {time()}")
                Price.append(bookOrder.price)
                Time.append(time())

            elif incomingOrder.remainingToFill < bookOrder.remainingToFill:  # book has greater volume
                volume = incomingOrder.remainingToFill
                incomingOrder.remainingToFill -= volume
                bookOrder.remainingToFill -= volume
                self.trades.append(Trade(
                    incomingOrder.side, bookOrder.price, volume, incomingOrder.order_id, bookOrder.order_id))
                print(f"PRICE IS {bookOrder.price} AT TIME {time()}")
                Price.append(bookOrder.price)
                Time.append(time())

                if bookOrder.side==Side.SELL:
                    self.asks.add(bookOrder)
                else:
                    self.bids.add(bookOrder)
                break

        if incomingOrder.remainingToFill > 0 and incomingOrder.__class__ == LimitOrder:
            if incomingOrder.side == Side.BUY:
                self.bids.add(incomingOrder)
            else:
                self.asks.add(incomingOrder)

    def getBid(self): return self.bids[0].price if len(self.bids)>0 else None
    def getAsk(self): return self.asks[0].price if len(self.asks)>0 else None

    def __repr__(self):
        #df = pd.DataFrame()
        lines = []
        lines.append("-"*5 + "OrderBook" + "-"*5)
        
        lines.append("\nAsks:")
        asks = self.asks.copy()
        while len(asks) > 0:
            lines.append(str(asks.pop()))
        #df['Ask'] = lines
        lines.append("\t"*5 + "Bids:")
        
        bids = list(reversed(self.bids.copy()))
        while len(bids) > 0:
            lines.append("\t"*5 + str(bids.pop()))
            #lines.append(str(bids.pop()))
        #df['Bid'] = lines
        lines.append("-"*20)
        #return df
        return "\n".join(lines)

    def __len__(self):
        return len(self.asks) + len(self.bids)

In [7]:
from random import getrandbits, randint, random

In [29]:
OB = Orderbook()
numOrders = 2**7
orders = []
for n in range(numOrders):
    if bool(getrandbits(1)):
        orders.append(LimitOrder(n, "BUY", randint(1, 200), randint(1, 4)))
    else:
        orders.append(LimitOrder(n, "SELL", randint(1, 200), randint(1, 4)))

from time import time
start = time()
for order in orders:
    OB.processOrder(order)
end = time()
totalTime = (end-start)

print("Time: " + str(totalTime))
print("Time per order (us): " + str(1000000*totalTime/numOrders))
print("Orders per second: " + str(numOrders/totalTime))


Time: 0.0013649463653564453
Time per order (us): 10.663643479347229
Orders per second: 93776.57851528384


# **NOW WE START THE TESTING PHASE**

In [30]:
order = LimitOrder(1, "BUY", 10, 100)
assert isinstance(order.order_id, int)
assert isinstance(order.time, int)
assert order.order_id == 1
assert order.side == "BUY"
assert order.size == 10
assert order.price == 100
print(order)

Limit Order: 10 unit(s) at 100.


In [31]:
order = MarketOrder(1, "SELL", 10)
assert isinstance(order.order_id, int)
assert isinstance(order.time, int)
assert order.order_id == 1
#assert order.side == Side.BUY
assert order.size == 10
print(order)

Market Order: 10 unit(s).


In [32]:
book = Orderbook()
#Book should be empty to begin with
assert len(book) == 0

assert book.getBid() == None
assert book.getAsk() == None
print(book)

-----OrderBook-----

Asks:
					Bids:
--------------------


In [33]:
book = Orderbook()
order = LimitOrder(0, Side.BUY, 10, 10)
book.processOrder(order)
# assert len(book) == 1
# assert book.getBid() == 10
# assert book.getAsk() == None
print(book)

-----OrderBook-----

Asks:
					Bids:
					Limit Order: 10 unit(s) at 10.
--------------------


In [34]:
book = Orderbook()
order = LimitOrder(0, Side.BUY, 10, 10)
book.processOrder(order)
order = LimitOrder(0,Side.SELL ,10, 12)
book.processOrder(order)

# assert len(book) == 0
# assert book.getBid() == None
# assert book.getAsk() == None
print(book)

-----OrderBook-----

Asks:
Limit Order: 10 unit(s) at 12.
					Bids:
					Limit Order: 10 unit(s) at 10.
--------------------


In [113]:
book = Orderbook()
order = LimitOrder(RandomIDGenerator('nis'), Side.SELL, 5, 105)
book.processOrder(order)
order = LimitOrder(RandomIDGenerator('nis'), Side.BUY, 5, 107)
book.processOrder(order)
order = LimitOrder(RandomIDGenerator('nis'), Side.BUY, 15, 105)
trade = book.processOrder(order)
order = LimitOrder(RandomIDGenerator('bus'), Side.BUY, 23, 105)
trade = book.processOrder(order)
order = LimitOrder(RandomIDGenerator('hum'), Side.SELL, 14, 105)
trade = book.processOrder(order)
order = LimitOrder(RandomIDGenerator('dep'), Side.BUY, 12, 105)
trade = book.processOrder(order)
order = LimitOrder(RandomIDGenerator('jay'), Side.SELL, 51, 103)
trade = book.processOrder(order)
order = LimitOrder(RandomIDGenerator('pro'), Side.BUY, 51, 107)
trade = book.processOrder(order)
order = LimitOrder(RandomIDGenerator('var'), Side.SELL, 1, 109)
trade = book.processOrder(order)
cancel_order = CancelOrder(10)
trade = book.processOrder(cancel_order)
order = MarketOrder(RandomIDGenerator('var'),Side.SELL,10)
order = book.processOrder(order)
order = LimitOrder(RandomIDGenerator('jam'), Side.BUY, 1, 109)
trade = book.processOrder(order)
order = LimitOrder(RandomIDGenerator('kari'), Side.SELL, 1, 109)
trade = book.processOrder(order)
order = LimitOrder(RandomIDGenerator('veru'), Side.BUY, 2, 109)
trade = book.processOrder(order)
order = LimitOrder(RandomIDGenerator('ku'), Side.SELL, 3, 109)
trade = book.processOrder(order)
order = LimitOrder(RandomIDGenerator('koo'), Side.SELL, 3, 107)
trade = book.processOrder(order)
# assert len(book) == 3
# assert book.getBid() == None
# assert book.getAsk() == 105
# assert len(book.trades)==1
print(book)

PRICE IS 105 AT TIME 1699035411.3235524
PRICE IS 105 AT TIME 1699035411.3238325
PRICE IS 105 AT TIME 1699035411.323857
PRICE IS 105 AT TIME 1699035411.3238964
PRICE IS 103 AT TIME 1699035411.323998
PRICE IS 107 AT TIME 1699035411.3242242
PRICE IS 109 AT TIME 1699035411.3244417
PRICE IS 109 AT TIME 1699035411.3245268
PRICE IS 107 AT TIME 1699035411.3246036
-----OrderBook-----

Asks:
Limit Order: 2 unit(s) at 109.
					Bids:
					Limit Order: 23 unit(s) at 107.
--------------------


In [114]:
orderbook = pd.DataFrame()
orderbook['Time_of_Execution'] = Trade_time_book
orderbook['Trades'] = Trade_book
orderbook['order_id'] = order_id

In [115]:
orderbook

,Time_of_Execution,Trades,order_id
0,1.699034e+09,Executed: Side.BUY 5 units at 105 ///@ id no. ...,71837051-nis
1,1.699034e+09,Executed: Side.SELL 14 units at 105 ///@ id no...,32463461-nis
2,1.699034e+09,Executed: Side.SELL 1 units at 105 ///@ id no....,32463461-nis
3,1.699034e+09,Executed: Side.SELL 23 units at 105 ///@ id no...,1365804-bus
4,1.699034e+09,Executed: Side.SELL 12 units at 105 ///@ id no...,64842218-dep
5,1.699034e+09,Executed: Side.BUY 15 units at 103 ///@ id no....,67461245-jay
6,1.699034e+09,Executed: Side.SELL 10 units at 107 ///@ id no...,14353971-pro
7,1.699034e+09,Executed: Side.BUY 1 units at 109 ///@ id no. ...,29932658-var
8,1.699034e+09,Executed: Side.BUY 1 units at 109 ///@ id no. ...,52407197-kari
9,1.699034e+09,Executed: Side.SELL 1 units at 109 ///@ id no....,96116120-veru


In [116]:
price_discovery = pd.DataFrame()
price_discovery['Price'] = Price
price_discovery['Time'] = Time
price_discovery

,Price,Time
0,105,1.699034e+09
1,105,1.699034e+09
2,105,1.699034e+09
3,105,1.699034e+09
4,103,1.699034e+09
...,...,...
76,103,1.699035e+09
77,107,1.699035e+09
78,109,1.699035e+09
79,109,1.699035e+09
